# Import Dependencies

In [50]:
# Dependencies
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import urllib.request
import requests
import pandas as pd
import time
import re
import json
import matplotlib.pyplot as plt
import numpy as np
import time
import warnings
from sqlalchemy import create_engine


# Import Hospital Infection Data

In [3]:
csv_hospital = "Healthcare_Associated_Infections-Hospital.csv"
hospital_df = pd.read_csv(csv_hospital)
hospital_df.head()

,Facility ID,Facility Name,Address,City,State,ZIP Code,County Name,Phone Number,Measure ID,Measure Name,Compared to National,Score,Footnote,Start Date,End Date
0,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_CILOWER,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,0.203,NaN,7/1/2019,12/31/2020
1,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_CIUPPER,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,1.23,NaN,7/1/2019,12/31/2020
2,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_DOPC,Central Line Associated Bloodstream Infection:...,No Different than National Benchmark,8631,NaN,7/1/2019,12/31/2020
3,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_ELIGCASES,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,9.008,NaN,7/1/2019,12/31/2020
4,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_NUMERATOR,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,5,NaN,7/1/2019,12/31/2020


# Review Data Frame

In [8]:
hospital_df.count()

Facility ID             174780
Facility Name           174780
Address                 174780
City                    174780
State                   174780
ZIP Code                174780
County Name             174780
Phone Number            174780
Measure ID              174780
Measure Name            174780
Compared to National    174780
Score                   174780
Footnote                 82415
Start Date              174780
End Date                174780
dtype: int64

In [15]:
#Footnotes column not needed. Drop column name 'Footnote' from Data Frame

hospital_df2 = hospital_df.drop(['Footnote'], axis = 1)



In [16]:
hospital_df2.count()

Facility ID             174780
Facility Name           174780
Address                 174780
City                    174780
State                   174780
ZIP Code                174780
County Name             174780
Phone Number            174780
Measure ID              174780
Measure Name            174780
Compared to National    174780
Score                   174780
Start Date              174780
End Date                174780
dtype: int64

In [22]:
#Count of records per state
hospital_df2.State.value_counts()

TX    14436
CA    12240
FL     6732
IL     6480
NY     6120
PA     6084
OH     6048
GA     4932
KS     4860
MI     4860
WI     4644
MN     4644
OK     4356
IN     4320
LA     4320
IA     4212
MO     4032
NC     3960
TN     3708
MS     3456
WA     3384
KY     3384
AL     3312
NE     3276
VA     3168
CO     3132
AZ     2988
AR     2844
NJ     2412
MA     2340
MT     2268
SC     2232
OR     2232
SD     2160
PR     1944
MD     1836
WV     1836
UT     1728
ND     1620
NM     1584
ID     1584
NV     1368
ME     1224
CT     1116
WY     1008
NH      936
HI      864
AK      828
VT      540
RI      396
DE      288
DC      288
VI       72
GU       72
AS       36
MP       36
Name: State, dtype: int64

In [28]:
#Find Unique Values for Column 'Compared to National'

print(hospital_df2['Compared to National'].unique())

['No Different than National Benchmark'
 'Better than the National Benchmark' 'Not Available'
 'Worse than the National Benchmark']


# Clean data frame

In [29]:
# Rename Columns

hospital_df3 = hospital_df2.rename(columns={'Facility ID':'facility_id',
                                 'Facility Name':'name',
                                 'Address':'address',
                                 'City':'city',
                                 'State':'state',
                                 'ZIP Code':'zip',
                                 'County Name':'county',
                                 'Phone Number':'phone',
                                 'Measure ID':'measure_id',
                                 'Measure Name':'measure_name',
                                 'Compared to National':'comparison',
                                 'Score':'score',
                                 'Start Date':'start',
                                 'End Date':'end'}
          )

In [30]:
hospital_df3.head()

,facility_id,name,address,city,state,zip,county,phone,measure_id,measure_name,comparison,score,start,end
0,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_CILOWER,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,0.203,7/1/2019,12/31/2020
1,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_CIUPPER,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,1.23,7/1/2019,12/31/2020
2,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_DOPC,Central Line Associated Bloodstream Infection:...,No Different than National Benchmark,8631,7/1/2019,12/31/2020
3,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_ELIGCASES,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,9.008,7/1/2019,12/31/2020
4,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_NUMERATOR,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,5,7/1/2019,12/31/2020


In [31]:
# Add an id column (the index)

hospital_df3["id"] = hospital_df3.index + 1

hospital_df3.head()

,facility_id,name,address,city,state,zip,county,phone,measure_id,measure_name,comparison,score,start,end,id
0,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_CILOWER,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,0.203,7/1/2019,12/31/2020,1
1,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_CIUPPER,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,1.23,7/1/2019,12/31/2020,2
2,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_DOPC,Central Line Associated Bloodstream Infection:...,No Different than National Benchmark,8631,7/1/2019,12/31/2020,3
3,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_ELIGCASES,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,9.008,7/1/2019,12/31/2020,4
4,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_NUMERATOR,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,5,7/1/2019,12/31/2020,5


In [32]:
# Set index
hospital_df3.set_index("id", inplace=True)

hospital_df3.head()

,facility_id,name,address,city,state,zip,county,phone,measure_id,measure_name,comparison,score,start,end
id,,,,,,,,,,,,,,
1,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_CILOWER,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,0.203,7/1/2019,12/31/2020
2,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_CIUPPER,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,1.23,7/1/2019,12/31/2020
3,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_DOPC,Central Line Associated Bloodstream Infection:...,No Different than National Benchmark,8631,7/1/2019,12/31/2020
4,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_ELIGCASES,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,9.008,7/1/2019,12/31/2020
5,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_NUMERATOR,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,5,7/1/2019,12/31/2020


In [33]:
# Need to Rename Start and End column... had trouble setting up the SQL table

hospital_df4 = hospital_df3.rename(columns={
                                 'start':'start_date',
                                 'end':'end_date'}
          )

In [46]:
hospital_df4.head()

,facility_id,name,address,city,state,zip,county,phone,measure_id,measure_name,comparison,score,start_date,end_date
id,,,,,,,,,,,,,,
1,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_CILOWER,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,0.203,7/1/2019,12/31/2020
2,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_CIUPPER,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,1.23,7/1/2019,12/31/2020
3,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_DOPC,Central Line Associated Bloodstream Infection:...,No Different than National Benchmark,8631,7/1/2019,12/31/2020
4,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_ELIGCASES,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,9.008,7/1/2019,12/31/2020
5,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_NUMERATOR,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,5,7/1/2019,12/31/2020


# Create the Database Connection

In [58]:
connection_string = "postgres:Rubenast09!@localhost:5432/hospital"
engine = create_engine(f'postgresql://{connection_string}')

In [59]:
# Confirm tables
engine.table_names()

<ipython-input-59-2d97e9123e98>:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  engine.table_names()


['hospital_data']

# Load DataFrame into database

In [60]:
hospital_df4.to_sql(name='hospital_data', con=engine, if_exists='append', index=True)

In [61]:
#Check to make sure that the data has been added by querying the hospital_data table

pd.read_sql_query('select * from hospital_data', con=engine).head()

,id,facility_id,name,address,city,state,zip,county,phone,measure_id,measure_name,comparison,score,start_date,end_date
0,1,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_CILOWER,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,0.203,7/1/2019,12/31/2020
1,2,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_CIUPPER,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,1.23,7/1/2019,12/31/2020
2,3,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_DOPC,Central Line Associated Bloodstream Infection:...,No Different than National Benchmark,8631,7/1/2019,12/31/2020
3,4,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_ELIGCASES,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,9.008,7/1/2019,12/31/2020
4,5,10001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,Houston,(334) 793-8701,HAI_1_NUMERATOR,Central Line Associated Bloodstream Infection ...,No Different than National Benchmark,5,7/1/2019,12/31/2020
